In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.oem'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request, url_check = f'''https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:{crawler_series['Part Number']}&size=1&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw''', crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_dict = resp.json()[0]['CrossReferences'] if 'CrossReferences' in resp.json()[0] else []
            list_reference = [dict_['reference'].strip() for dict_ in list_dict if dict_['referenceType'].strip() == 'OE Part']
            oem = ';'.join(list(dict.fromkeys(list_reference)))

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Type': crawler_series['Type'],
                                          'Part Number': crawler_series['Part Number'],
                                          'Title': crawler_series['Title'],
                                          'Application Summary': crawler_series['Application Summary'],
                                          'OEM': oem,
                                          'Picture': crawler_series['Picture'],
                                          'Url': crawler_series['Url'],
                                          'Subtitle': crawler_series['Subtitle'],
                                          'Json_Src': crawler_series['Json_Src'],
                                          'Json_Info': crawler_series['Json_Info']})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-oem_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/25 17:00:29] - 2. https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10000&size=1&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/25 17:00:29] - 3. https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10005&size=1&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/25 17:00:29] - 5. https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10007&size=1&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/25 17:00:29] - 4. https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:40-10006&size=1&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3557.51it/s]

Done ~
